In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import OneClassSVM
import numpy as np

### Load MVTec Transistor Dataset

In [6]:
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"atalaypabuu","key":"1a6cc847a55e48a8f9166f9dc2b7ce9b"}'}

In [7]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download -d 'leezhixiong/mvtec-transistor-dataset'

 99% 379M/384M [00:04<00:00, 103MB/s] 
100% 384M/384M [00:04<00:00, 82.6MB/s]


In [9]:
import zipfile

# Unzip the downloaded file
def unzip_data(zip_file_name):
  zip_ref = zipfile.ZipFile(zip_file_name, "r")
  zip_ref.extractall()
  zip_ref.close()

In [10]:
unzip_data('/content/mvtec-transistor-dataset.zip')

In [17]:
DATASET_PATH = '/content/transistor'
DATASET_TRAINING_PATH = '/content/transistor/train/good'
DATASET_TEST_PATH = '/content/transistor/test'
DATASET_TEST_GOOD_PATH = '/content/transistor/test/good'
DATASET_TEST_MISPLACED_PATH = '/content/transistor/test/misplaced'
DATASET_TEST_CUTLEAD_PATH = '/content/transistor/test/cut_lead'
DATASET_TEST_DMG_PATH = '/content/transistor/test/damaged_case'
MODEL_CHECKPOINT_PATH = '/content/base_model_checkpoint/cp.ckpt'

In [12]:
import os

def walk_through_dir(data_dir):
  for dirpath, dirnames, filenames in os.walk(data_dir):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [13]:
walk_through_dir(DATASET_PATH)

There are 3 directories and 2 images in '/content/transistor'.
There are 1 directories and 0 images in '/content/transistor/train'.
There are 0 directories and 213 images in '/content/transistor/train/good'.
There are 5 directories and 0 images in '/content/transistor/test'.
There are 0 directories and 10 images in '/content/transistor/test/misplaced'.
There are 0 directories and 10 images in '/content/transistor/test/bent_lead'.
There are 0 directories and 10 images in '/content/transistor/test/damaged_case'.
There are 0 directories and 60 images in '/content/transistor/test/good'.
There are 0 directories and 10 images in '/content/transistor/test/cut_lead'.
There are 4 directories and 0 images in '/content/transistor/ground_truth'.
There are 0 directories and 10 images in '/content/transistor/ground_truth/misplaced'.
There are 0 directories and 10 images in '/content/transistor/ground_truth/bent_lead'.
There are 0 directories and 10 images in '/content/transistor/ground_truth/damaged

### Prepare MVTec Transistor Dataset

In [14]:
from tensorflow import keras
import glob   # In order to get images as matrices from directory

def get_imgs_from_directory_as_numpy_array(dir, ext, size=None, scl=None):
    formats = ['jpg', 'png', 'jpeg']

    if ext in formats:
        rgx_for_img = dir + "/*." + ext
        img_list = glob.glob(rgx_for_img)
    def process_images(filename):
        img = keras.preprocessing.image.load_img(filename, color_mode="rgb", target_size=size)
        img_arr = keras.preprocessing.image.img_to_array(img)
        img_arr = np.array(img_arr).astype('float32')
        if scl == '8bit': img_arr /= 255.0
        return img_arr

    return np.array(list(map(process_images, img_list)))
    # https://keras.io/api/preprocessing/image/
    # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [19]:
IMG_SIZE = (224, 224)   # For VGG16 Input
IMG_SHAPE = (224, 224, 3)

In [18]:
x_train_transistor = get_imgs_from_directory_as_numpy_array(DATASET_TRAINING_PATH, "png", size=IMG_SIZE, scl='8bit')
x_test_good = get_imgs_from_directory_as_numpy_array(DATASET_TEST_GOOD_PATH, "png", size=IMG_SIZE, scl='8bit')
x_test_misplaced = get_imgs_from_directory_as_numpy_array(DATASET_TEST_MISPLACED_PATH, "png", size=IMG_SIZE, scl='8bit')
x_test_cutlead = get_imgs_from_directory_as_numpy_array(DATASET_TEST_CUTLEAD_PATH, "png", size=IMG_SIZE, scl='8bit')
x_test_damaged = get_imgs_from_directory_as_numpy_array(DATASET_TEST_DMG_PATH, "png", size=IMG_SIZE, scl='8bit')

print("x_train_transistor.shape: ", x_train_transistor.shape)
print("x_test_good.shape: ", x_test_good.shape)
print("x_test_misplaced.shape: ", x_test_misplaced.shape)
print("x_test_cutlead.shape: ", x_test_cutlead.shape)
print("x_test_damaged.shape: ", x_test_damaged.shape)

x_train_transistor.shape:  (213, 224, 224, 3)
x_test_good.shape:  (60, 224, 224, 3)
x_test_misplaced.shape:  (10, 224, 224, 3)
x_test_cutlead.shape:  (10, 224, 224, 3)
x_test_damaged.shape:  (10, 224, 224, 3)


### Load the pre-trained MobileNetV2 model (without classification head)

In [20]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [28]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                            

In [21]:
# Add a global average pooling layer to reduce the spatial dimensions
glb_avg_layer = GlobalAveragePooling2D()(base_model.output)

### Create a feature extraction model using the pre-trained MobileNetV2

In [22]:
feature_extraction_model = tf.keras.Model(inputs=base_model.input, outputs=glb_avg_layer)

### Extract deep features for target class and outliers

In [23]:
x_train_features = feature_extraction_model.predict(x_train_transistor)
x_test_misplaced_features = feature_extraction_model.predict(x_test_misplaced)  # Outliers

1/1 [==============================] - 1s 513ms/step


### Train One-Class SVM

In [24]:
# Combine the target class features with some proportion of outlier features to create the training set
# proportion_outliers = 0.1
# train_features = np.vstack((target_class_features, outlier_features[:int(num_outliers * proportion_outliers)]))

=== np.vstack ===

a = np.array([1, 2, 3])

b = np.array([4, 5, 6])

np.vstack((a,b))

Output: array([[1, 2, 3],
       [4, 5, 6]])

=== np.hstack ===

a = np.array((1,2,3))

b = np.array((4,5,6))

np.hstack((a,b))

Output: array([1, 2, 3, 4, 5, 6])

=======================

a = np.array([[1],[2],[3]])

b = np.array([[4],[5],[6]])

np.hstack((a,b))

Output: array([[1, 4],
       [2, 5],
       [3, 6]])

### Fit the One-Class SVM model

class sklearn.svm.OneClassSVM(*, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, nu=0.5, shrinking=True, cache_size=200, verbose=False, max_iter=-1)[source]

In [25]:
ocsvm = OneClassSVM(gamma='auto')  # You can adjust the hyperparameters as needed
ocsvm.fit(x_train_features)

OneClassSVM(gamma='auto')

In [41]:
x_test_misplaced.shape

(10, 224, 224, 3)

In [42]:
x_test_misplaced_features.shape

(10, 1280)

In [43]:
# Test on some new data (you can use a separate test set or additional data)
# For example, using only the misplaced (outlier) data for testing
predictions = ocsvm.predict(x_test_misplaced_features)

In [44]:
# In the predictions, -1 indicates outliers, and 1 indicates the target class
print("Predictions:", predictions)

Predictions: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [45]:
x_test_good_features = feature_extraction_model.predict(x_test_good)
x_test_cutlead_features = feature_extraction_model.predict(x_test_cutlead)  # Outliers
x_test_damaged_features = feature_extraction_model.predict(x_test_damaged)  # Outliers

1/1 [==============================] - 0s 29ms/step


In [46]:
x_test_good_ocsvm_predict = ocsvm.predict(x_test_good_features)
x_test_cutlead_ocsvm_predict = ocsvm.predict(x_test_cutlead_features)
x_test_damaged_ocsvm_predict = ocsvm.predict(x_test_damaged_features)

In [49]:
# In the predictions, -1 indicates outliers, and 1 indicates the target class
print("Good Transistor Predictions:", x_test_good_ocsvm_predict)
print("Cutlead Anomlay Predictions:", x_test_cutlead_ocsvm_predict)
print("Damaged Case Anomaly Predictions:", x_test_damaged_ocsvm_predict)

Good Transistor Predictions: [-1 -1  1  1 -1 -1 -1 -1 -1  1  1  1 -1 -1  1  1 -1  1  1  1  1 -1  1 -1
  1  1 -1 -1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1 -1  1  1  1  1 -1  1 -1  1
 -1  1 -1  1  1 -1 -1  1  1 -1  1  1]
Cutlead Anomlay Predictions: [-1 -1 -1  1 -1 -1 -1 -1 -1 -1]
Damaged Case Anomaly Predictions: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
